<a href="https://colab.research.google.com/github/andohyung/Likelion_Project2/blob/main/%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D_%EB%A9%B4%EC%A0%91%EB%8B%B5%EB%B3%80%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JSON 파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import json
import pandas as pd

In [ ]:


# JSON 파일들이 저장된 상위 폴더 경로 지정
json_folder_path = '/content/drive/MyDrive/멋사_실전프로젝트2/전체면접데이터'  # 상위 폴더 경로를 입력하세요.

# 폴더 내 모든 JSON 파일을 읽고 질문과 답변 추출
data_list = []

# 상위 폴더 내의 모든 하위 폴더를 재귀적으로 탐색하며 JSON 파일 읽기
for root, dirs, files in os.walk(json_folder_path):
    for filename in files:
        if filename.endswith('.json'):
            file_path = os.path.join(root, filename)  # 파일 경로 생성

            # JSON 파일 열기
            with open(file_path, 'r', encoding='utf-8') as f:
                try:
                    json_data = json.load(f)

                    # 데이터에서 질문과 답변 추출
                    question_text = json_data["dataSet"]["question"]["raw"]["text"]
                    answer_text = json_data["dataSet"]["answer"]["raw"]["text"]

                    # 질문과 답변을 결합하여 input_text 생성
                    input_text = question_text + "</s>" + answer_text
                    data_list.append({"input_text": input_text})

                except json.JSONDecodeError as e:
                    # JSONDecodeError 발생 시 로그를 남기고 파일 건너뛰기
                    print(f"JSONDecodeError 발생: 파일 {file_path} - {e}")
                except KeyError as e:
                    # KeyError 발생 시 로그를 남기고 파일 건너뛰기
                    print(f"KeyError 발생: 파일 {file_path} - {e}")
                except Exception as e:
                    # 기타 예외 발생 시 로그를 남기고 파일 건너뛰기
                    print(f"기타 오류 발생: 파일 {file_path} - {e}")

# DataFrame으로 변환
data_df = pd.DataFrame(data_list)
print(data_df.head())



JSONDecodeError 발생: 파일 /content/drive/MyDrive/멋사_실전프로젝트2/전체면접데이터/TL_05.ICT_Male_New/ckmk_d_ict_m_n_337139.json - Expecting property name enclosed in double quotes: line 22 column 21 (char 576)
JSONDecodeError 발생: 파일 /content/drive/MyDrive/멋사_실전프로젝트2/전체면접데이터/TL_01.Management_Female_New/ckmk_d_bm_f_n_343176.json - Invalid control character at: line 23 column 433 (char 1029)
JSONDecodeError 발생: 파일 /content/drive/MyDrive/멋사_실전프로젝트2/전체면접데이터/TL_01.Management_Female_New/ckmk_d_bm_f_n_269424.json - Invalid control character at: line 23 column 571 (char 1132)
JSONDecodeError 발생: 파일 /content/drive/MyDrive/멋사_실전프로젝트2/전체면접데이터/TL_01.Management_Female_New/ckmk_d_bm_f_n_178194.json - Invalid control character at: line 23 column 135 (char 755)
                                          input_text
0  상사가 회사의 이익에 반하는 행동을 하는 것을 직접 목격했습니다 그때 어떻게 행동하...
1  요즘 아이시티 산업의 인문학적 소양을 많이 접목시키는데요 아이시티 업무의 인문학적 ...
2  만약 이 면접에서 최종

In [ ]:
# DataFrame을 CSV 파일로 저장하기
output_path = '/content/drive/MyDrive/멋사_실전프로젝트2/전체_면접_데이터.csv'  # 원하는 저장 경로를 입력하세요.
data_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"DataFrame이 {output_path}에 성공적으로 저장되었습니다.")

DataFrame이 /content/drive/MyDrive/멋사_실전프로젝트2/전체_면접_데이터.csv에 성공적으로 저장되었습니다.


In [ ]:
# 저장한 CSV 파일 다시 읽기
saved_df = pd.read_csv('/content/drive/MyDrive/멋사_실전프로젝트2/전체_면접_데이터.csv', encoding='utf-8-sig')

# 저장된 파일의 앞부분 출력
print(saved_df.head())

                                          input_text
0  상사가 회사의 이익에 반하는 행동을 하는 것을 직접 목격했습니다 그때 어떻게 행동하...
1  요즘 아이시티 산업의 인문학적 소양을 많이 접목시키는데요 아이시티 업무의 인문학적 ...
2  만약 이 면접에서 최종 탈락하게 되신다면 어떤 점이 부족해서 탈락했다고 생각하실 것...
3  지원자께서 업무를 하다 보면은 업무량이 과도하게 많아질 수가 있는데요 그러한 상황에...
4  운행하던 기차가 산사태로 삼십 분 동안 정차되었습니다 이때 지원자님께서 관리 책임의...


모델 및 토크나이저

In [ ]:
#!pip install transformers torch

import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# Step 3: KoGPT2 토크나이저 및 모델 설정
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>'
)

# KoGPT2 모델 불러오기
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# GPU 사용 설정 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

데이터셋 데이터로더

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Step 5: Custom Dataset 정의
class InterviewDataset(Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs['input_ids']
        self.attention_mask = inputs['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

# 데이터프레임에서 텍스트 데이터 추출 및 토크나이징
texts = saved_df['input_text'].tolist()  # 'text_column_name'은 실제 텍스트 열 이름으로 대체하세요.

# 토크나이저를 사용해 텍스트 데이터 처리
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


# 전체 데이터셋 생성
dataset = InterviewDataset(inputs)

# 데이터셋 크기 확인
dataset_size = len(dataset)

# 데이터셋을 80% Train, 10% Validation, 10% Test로 나누기
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# random_split을 이용해 데이터셋 분할
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# DataLoader 정의 (배치 크기 설정)
batch_size = 16  # 배치 크기는 상황에 따라 조정 가능

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 데이터로더 확인
print(f'Train DataLoader size: {len(train_dataloader)} batches')
print(f'Validation DataLoader size: {len(val_dataloader)} batches')
print(f'Test DataLoader size: {len(test_dataloader)} batches')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Train DataLoader size: 3404 batches
Validation DataLoader size: 426 batches
Test DataLoader size: 426 batches


옵티마이저

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup


# Step 6: Optimizer 및 Scheduler 설정
optimizer = AdamW(model.parameters(), lr=0.0001)

epochs = 3
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

파인튜닝

In [ ]:
!pip install tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
import torch.nn.functional as F
from tqdm import tqdm

# Step 7: 모델 훈련 루프
model.train()  # 학습 모드로 설정

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')

    # tqdm을 사용해 DataLoader의 진행률을 표시
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")

    for batch in progress_bar:
        # 입력 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # 모델의 그라디언트를 초기화
        model.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # 진행률 막대에 현재 Loss 표시
        progress_bar.set_postfix(loss=loss.item())

        # Backward pass 및 옵티마이저를 통해 가중치 업데이트
        loss.backward()
        optimizer.step()
        scheduler.step()




Epoch 1/3


Epoch 1/3: 100%|██████████| 3404/3404 [51:07<00:00,  1.11batch/s, loss=1.37]


Epoch 2/3


Epoch 2/3: 100%|██████████| 3404/3404 [51:09<00:00,  1.11batch/s, loss=1.45]


Epoch 3/3


Epoch 3/3: 100%|██████████| 3404/3404 [51:11<00:00,  1.11batch/s, loss=1.44]


In [ ]:
# Step 8: 훈련된 모델 저장
save_path = '/content/drive/MyDrive/멋사_실전프로젝트2/lr수정_finetuned_model.pth'  # 원하는 경로로 수정
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f'Model saved to {save_path}')

Model saved to /content/drive/MyDrive/멋사_실전프로젝트2/lr수정_finetuned_model.pth


In [ ]:
pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 저장된 모델 경로
model_path = '/content/drive/MyDrive/멋사_실전프로젝트2/lr수정_finetuned_model.pth'

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')

# 저장된 가중치 로드
model.load_state_dict(torch.load(model_path))
model.eval()  # 평가 모드로 설정

<ipython-input-23-25349a7a3d8f>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 예제 입력
prompt = "질문: 면접 시 가장 중요한 요소는 무엇인가요? 답변:"

# 모델로 생성한 텍스트
generated_text = generate_text(prompt, model, tokenizer)

# 출력
print("Generated Text:", generated_text)

# BERTScore 비교
references = ["면접에서 가장 중요한 것은 신뢰와 준비입니다."]
candidates = [generated_text]
P, R, F1 = score(candidates, references, lang="ko", verbose=True)

print(f"Precision: {P.mean().item():.4f}")
print(f"Recall: {R.mean().item():.4f}")
print(f"F1 Score: {F1.mean().item():.4f}")


Generated Text: 질문: 면접 시 가장 중요한 요소는 무엇인가요? 답변: 면접자 질문 네 면접자 질문 네 면접자 질문 네 면접자 질문 네 면접자 질문 네 면접자 질문 네. 저는 면접을 준비할 때 가장 중요한 것은 바로 그


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.35 seconds, 2.85 sentences/sec
Precision: 0.6147
Recall: 0.7613
F1 Score: 0.6802
